In [34]:
from ortools.sat.python.cp_model import CpModel, CpSolver, IntVar, OPTIMAL, FEASIBLE
from pysat.solvers import Solver as SATSolver
import gurobipy as gp
import networkx as nx
import itertools
import matplotlib.pyplot as plt
import math

from typing import List

import random

In [35]:
random.seed(1234568)
ints = [random.randint(1, 20) for _ in range(69)]
ints

[6,
 16,
 1,
 15,
 11,
 13,
 16,
 8,
 16,
 5,
 8,
 1,
 17,
 10,
 19,
 13,
 16,
 19,
 16,
 5,
 17,
 4,
 11,
 8,
 4,
 3,
 16,
 2,
 20,
 20,
 1,
 12,
 4,
 20,
 1,
 18,
 11,
 10,
 5,
 12,
 10,
 5,
 4,
 13,
 11,
 5,
 18,
 15,
 13,
 9,
 1,
 18,
 15,
 4,
 16,
 11,
 17,
 3,
 2,
 16,
 15,
 6,
 17,
 14,
 11,
 3,
 14,
 17,
 5]

In [36]:
m = CpModel()
n = 3

# create container variables
partitions: List[List[IntVar]] = [[m.NewBoolVar(f"int_{i}_part_of_partition_{p}") for i in range(len(ints))] for p in range(n)]

# break symmetry
m.Add(partitions[0][0] == 1)

# only allow each int to be stored in exactly one container
for i in range(len(ints)):
    m.AddExactlyOne([p[i] for p in partitions])

# build the sums within each container
sums = [sum(p[i] * ints[i] for i in range(len(ints))) for p in partitions]

# force pairwise equality of sums
for sum1, sum2 in itertools.combinations(sums, 2):
    m.Add(sum1 == sum2)

solver = CpSolver()
status = solver.Solve(m)
print(solver.ResponseStats())

if status in (OPTIMAL, FEASIBLE):
    for p in partitions:
        values = [i for (i, v) in zip(ints, p) if solver.BooleanValue(v)]
        print(f"sum({sorted(values)}) = {sum(values)}")
else:
    print("Unsatisfiable!")




CpSolverResponse summary:
status: OPTIMAL
objective: 0
best_bound: 0
booleans: 204
conflicts: 0
branches: 408
propagations: 408
integer_propagations: 816
restarts: 408
lp_iterations: 113
walltime: 0.0191478
usertime: 0.0191478
deterministic_time: 0.0115339
gap_integral: 0

sum([1, 1, 2, 4, 4, 5, 6, 8, 10, 11, 11, 12, 13, 13, 13, 14, 15, 16, 16, 17, 17, 17, 20]) = 246
sum([1, 1, 3, 3, 4, 5, 5, 5, 6, 8, 9, 11, 11, 11, 12, 15, 16, 16, 16, 16, 17, 18, 18, 19]) = 246
sum([1, 2, 3, 4, 4, 5, 5, 8, 10, 10, 11, 13, 14, 15, 15, 16, 16, 17, 18, 19, 20, 20]) = 246
